In [1]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import random
import unicodedata

In [2]:


# Load dataset
df = pd.read_csv("/content/Roman-Urdu-Poetry.csv")  # Use the correct file name

def clean_text(text):
    if pd.isna(text):
        return ""

    # Normalize Unicode characters (removes accents and diacritics)
    text = ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

    # Remove unwanted characters except for basic punctuation
    text = re.sub(r"[^a-zA-Z0-9\s.,?!]", "", text)

    # Remove dots within words (fix ja.ega -> jaega, ro.ega -> roega)
    text = re.sub(r"\.(?=\w)", "", text)

    # Replace multiple spaces and newlines with a single space
    text = re.sub(r"\s+", " ", text).strip()

    # Convert to lowercase
    text = text.lower()

    return text

# Apply cleaning function to poetry column
df["Poetry"] = df["Poetry"].apply(clean_text)


In [5]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(df['Poetry'])
sequences = tokenizer.texts_to_sequences(df['Poetry'])
max_sequence_length = 20

input_sequences = []
for seq in sequences:
    for i in range(1, min(len(seq), max_sequence_length)):
        input_sequences.append(seq[:i+1])

input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [6]:
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length-1),
    Bidirectional(LSTM(128, return_sequences=True)),
    LSTM(128),
    Dense(128, activation='relu'),
    Dense(len(tokenizer.word_index) + 1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:

model.fit(X, y, epochs=100, verbose=1)
model.save("poetry_model.h5")

Epoch 1/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.0430 - loss: 7.5122
Epoch 2/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.0462 - loss: 6.4869
Epoch 3/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.0432 - loss: 6.2877
Epoch 4/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.0453 - loss: 6.1853
Epoch 5/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0511 - loss: 6.0793
Epoch 6/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.0618 - loss: 6.0167
Epoch 7/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0649 - loss: 5.8911
Epoch 8/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.0734 - loss: 5.8011
Epoch 9/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.0856 - loss: 5.6704
Epoch 10/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0943 - loss: 5.5969
Epoch 11/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.0984 - loss: 5.4781
Epoch 12/100
779/77

In [10]:
import numpy as np
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_poem(prompt, num_lines, words_per_line, temperature):
    """
    Generates a poem based on the given prompt.

    Parameters:
    - prompt (str): The initial word or phrase to start the poem.
    - num_lines (int): Number of lines in the generated poem.
    - words_per_line (int): Number of words per line.
    - temperature (float): Controls the randomness of predictions.

    Returns:
    - str: The generated poem.
    """
    poem = []
    current_word = prompt.lower()

    for _ in range(num_lines):
        line = current_word  # Start each line with the prompt word

        for _ in range(words_per_line - 1):
            token_list = tokenizer.texts_to_sequences([line])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')

            predictions = model.predict(token_list, verbose=0)[0]
            predictions = np.log(predictions + 1e-10) / temperature
            exp_preds = np.exp(predictions)
            predictions = exp_preds / np.sum(exp_preds)

            sorted_indices = np.argsort(predictions)[-5:]  # Top 5 words
            possible_words = [tokenizer.index_word.get(idx, None) for idx in sorted_indices if idx in tokenizer.index_word]
            possible_words = [word for word in possible_words if word is not None]

            if possible_words:
                word = random.choices(possible_words, weights=predictions[sorted_indices])[0]
            else:
                break

            line += " " + word
            current_word = word

        poem.append(line.capitalize())

    return "\n".join(poem)

# Get user input
prompt = input("Enter the prompt for the poem: ")
num_lines = int(input("Enter the number of lines: "))
words_per_line = int(input("Enter the number of words per line: "))
temperature = float(input("Enter the temperature (e.g., 0.8 for creativity): "))

# Generate and print the poem
poem = generate_poem(prompt, num_lines, words_per_line, temperature)
print("\nGenerated Poem:\n")
print(poem)


Enter the prompt for the poem: tere husn k naam 
Enter the number of lines: 4
Enter the number of words per line: 5
Enter the temperature (e.g., 0.8 for creativity): 0.8

Generated Poem:

Tere husn k naam  ja raha hai nikharta
Nikharta se duur kho jaegi
Jaegi khamushi ki khushbu khuli
Khuli ankhon men sapna jhankta


In [20]:
import gradio as gr
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer

# Load trained model
model = load_model("poetry_model.h5")

df = pd.read_csv("/content/Roman-Urdu-Poetry.csv")  # Use the correct file name

def clean_text(text):
    if pd.isna(text):
        return ""

    # Normalize Unicode characters (removes accents and diacritics)
    text = ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

    # Remove unwanted characters except for basic punctuation
    text = re.sub(r"[^a-zA-Z0-9\s.,?!]", "", text)

    # Remove dots within words (fix ja.ega -> jaega, ro.ega -> roega)
    text = re.sub(r"\.(?=\w)", "", text)

    # Replace multiple spaces and newlines with a single space
    text = re.sub(r"\s+", " ", text).strip()

    # Convert to lowercase
    text = text.lower()

    return text

# Apply cleaning function to poetry column
df["Poetry"] = df["Poetry"].apply(clean_text)

# Define sequence length
max_sequence_length = 20  # Ensure it matches training
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(df['Poetry'])
sequences = tokenizer.texts_to_sequences(df['Poetry'])
max_sequence_length = 20

input_sequences = []
for seq in sequences:
    for i in range(1, min(len(seq), max_sequence_length)):
        input_sequences.append(seq[:i+1])

input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

def generate_poem(prompt, num_lines, words_per_line, temperature):
    poem = []
    current_word = prompt.lower()

    for _ in range(num_lines):
        line = current_word

        for _ in range(words_per_line - 1):
            token_list = tokenizer.texts_to_sequences([line])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')

            predictions = model.predict(token_list, verbose=0)[0]
            predictions = np.log(predictions + 1e-10) / temperature
            exp_preds = np.exp(predictions)
            predictions = exp_preds / np.sum(exp_preds)

            sorted_indices = np.argsort(predictions)[-5:]
            possible_words = [tokenizer.index_word.get(idx, None) for idx in sorted_indices if idx in tokenizer.index_word]
            possible_words = [word for word in possible_words if word is not None]

            if possible_words:
                word = random.choices(possible_words, weights=predictions[sorted_indices])[0]
            else:
                break

            line += " " + word
            current_word = word

        poem.append(line.capitalize())

    return "\n".join(poem)

# Define Gradio interface
iface = gr.Interface(
    fn=generate_poem,
    inputs=[
        gr.Textbox(label="Enter Prompt"),
        gr.Number(label="Number of Lines", value=4, precision=0),
        gr.Number(label="Words per Line", value=5, precision=0),
        gr.Slider(0.1, 2.0, value=0.8, label="Temperature")
    ],
    outputs=gr.Textbox(label="Generated Poem"),
    title="Roman Urdu Poetry Generator",
    description="Generate poetry based on a given prompt using an LSTM model."
)

# Launch app
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://13c9701246c5f3416a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
